## 질문 유형별 도구 자동 선택 로직 구현

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

In [2]:
# Filename: routed_tool_agent.py
# LangChain 1.0 기준 예제 (2025-10)
# 실행 전:  pip install langchain langchain-core langchain-openai langgraph pydantic
# 환경변수:  export OPENAI_API_KEY=...  (Windows: set OPENAI_API_KEY=...)

from __future__ import annotations
from typing import Literal, TypedDict, List, Optional
from pydantic import BaseModel, Field
import operator
import ast

from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

########################################
# 0) LLM 설정
########################################
# 필요 시 모델 변경 가능: "gpt-4o-mini", "gpt-4o", "gpt-4.1-mini" 등
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

########################################
# 1) 도구 정의
########################################

# 1-1) 안전한 사칙연산 계산기
#      ast를 이용해 + - * / ( ) 및 정수/실수만 허용한다.
# allowed_nodes = {
#     ast.Expression, ast.BinOp, ast.UnaryOp, ast.Num, ast.Constant, ast.Load,
#     ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.USub, ast.UAdd, ast.Mod,
#     ast.FloorDiv, ast.LParen if hasattr(ast, "LParen") else ast.AST,  # 호환성
# }

allowed_nodes = {
    ast.Expression, ast.BinOp, ast.UnaryOp, ast.Constant, ast.Load,
    ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.USub, ast.UAdd, ast.Mod,
    ast.FloorDiv,
}

def _safe_eval(expr: str) -> float:
    def _check(node):
        if type(node) not in allowed_nodes:
            raise ValueError(f"허용되지 않는 표현: {type(node).__name__}")
        for child in ast.iter_child_nodes(node):
            _check(child)

    node = ast.parse(expr, mode="eval")
    _check(node)
    return eval(compile(node, "<expr>", "eval"), {"__builtins__": {}})

@tool
def calculator(expression: str) -> str:
    """간단한 수학 계산을 수행한다. 예: '12 * (3 + 4) / 7'"""
    try:
        value = _safe_eval(expression)
        return f"계산 결과: {value}"
    except Exception as e:
        return f"계산 실패: {e}"

# 1-2) 로컬 FAQ 검색 도구 (샘플용 간단 키워드 매칭)
_FAQ = [
    ("OpenAI API 키 설정", "환경변수 OPENAI_API_KEY를 설정한다."),
    ("LangChain 버전 확인", "import langchain; print(langchain.__version__)을 사용한다."),
    ("RAG 개념", "내 문서를 벡터화해 검색하고 LLM이 응답을 생성하는 방식이다."),
    ("LangGraph", "상태 기반 그래프 실행으로 노드/엣지로 워크플로우를 정의한다."),
]

@tool
def local_faq_search(query: str) -> str:
    """샘플 FAQ에서 질의어가 포함된 항목을 찾아 요약한다."""
    q = query.lower()
    hits = [f"- {k}: {v}" for k, v in _FAQ if q in k.lower() or q in v.lower()]
    if not hits:
        return "관련 FAQ가 없다."
    return "FAQ 검색 결과:\n" + "\n".join(hits)

# 1-3) 로컬 키워드 검색 도구 (샘플 코퍼스)
_DOCS = [
    "LangChain 1.0에서는 import 경로가 크게 변경되었다.",
    "LangGraph는 분기/루프/중단을 다루는 에이전트 워크플로우에 적합하다.",
    "OpenAI의 함수 호출을 통해 도구를 자동 선택하도록 만들 수 있다.",
    "RAG 파이프라인은 로더→청크→임베딩→벡터DB→retriever→LLM 순서로 구성한다.",
]

@tool
def local_keyword_search(query: str) -> str:
    """간단한 로컬 문서 키워드 검색을 수행한다."""
    q = query.lower()
    hits = [f"- {t}" for t in _DOCS if q in t.lower()]
    if not hits:
        return "검색 결과가 없다."
    return "로컬 검색 결과:\n" + "\n".join(hits)

# 필요한 경우 외부 웹검색 도구를 교체/추가할 수 있다.
# 예: tavily / ddg / requests 기반 커스텀 검색 등

TOOLS = [calculator, local_faq_search, local_keyword_search]

########################################
# 2) 질문 분류 모델 (구조화 출력)
########################################

class IntentSchema(BaseModel):
    intent: Literal[
        "math",        # 수학/계산
        "faq",         # FAQ/설정/개념 질의
        "search",      # 일반 정보 검색
        "chitchat",    # 잡담/일반 설명
    ] = Field(description="사용자 질문의 유형")
    reason: str = Field(description="이 분류에 도달한 근거 한 줄")

# LLM에게 구조화 출력 강제
classifier = llm.with_structured_output(IntentSchema)

########################################
# 3) 그래프 상태 및 노드 함수
########################################

class AgentState(TypedDict):
    question: str
    intent: Optional[str]
    reason: Optional[str]
    tool_result: Optional[str]
    final_answer: Optional[str]

def classify_node(state: AgentState) -> AgentState:
    """질문을 Intent로 분류한다."""
    res = classifier.invoke(
        f"질문 유형을 분류하라.\n질문: {state['question']}\n"
        "가능한 의도: math, faq, search, chitchat\n"
        "규칙:\n"
        "- 사칙연산/수치 계산이면 math\n"
        "- 설정/버전/개념/가이드면 faq\n"
        "- 배경지식/일반정보 찾기는 search\n"
        "- 그 외는 chitchat"
    )
    state["intent"] = res.intent
    state["reason"] = res.reason
    return state

def math_node(state: AgentState) -> AgentState:
    """계산기 도구 체인 실행."""
    expr = state["question"]
    result = calculator.invoke({"expression": expr})
    state["tool_result"] = result
    return state

def faq_node(state: AgentState) -> AgentState:
    """FAQ 도구 체인 실행."""
    result = local_faq_search.invoke({"query": state["question"]})
    state["tool_result"] = result
    return state

def search_node(state: AgentState) -> AgentState:
    """로컬 키워드 검색 도구 체인 실행."""
    result = local_keyword_search.invoke({"query": state["question"]})
    state["tool_result"] = result
    return state

def chitchat_node(state: AgentState) -> AgentState:
    """도구 없이 LLM 직접 응답."""
    answer = llm.invoke(
        "다음 질문에 간결하게 한국어로 답하라.\n"
        "종결은 ~이다/~한다로 한다.\n"
        f"질문: {state['question']}"
    ).content
    state["tool_result"] = answer
    return state

def compose_node(state: AgentState) -> AgentState:
    """최종 응답 생성. 도구 결과와 분류 근거를 바탕으로 정리."""
    prompt = f"""
다음 내용을 바탕으로 최종 답변을 한국어로 정리하라.
- 종결은 ~이다/~한다로 한다.
- 불필요한 장식은 하지 않는다.
- 핵심 먼저, 필요한 경우 간단한 단계/코드/다음 행동을 제시한다.

질문: {state['question']}
분류: {state.get('intent')}
근거: {state.get('reason')}
도구결과:
{state.get('tool_result') or '없음'}
"""
    state["final_answer"] = llm.invoke(prompt).content
    return state

########################################
# 4) 그래프 구성
########################################

graph = StateGraph(AgentState)

graph.add_node("classify", RunnableLambda(classify_node))
graph.add_node("math", RunnableLambda(math_node))
graph.add_node("faq", RunnableLambda(faq_node))
graph.add_node("search", RunnableLambda(search_node))
graph.add_node("chitchat", RunnableLambda(chitchat_node))
graph.add_node("compose", RunnableLambda(compose_node))

def route_from_intent(state: AgentState) -> str:
    intent = state.get("intent")
    if intent == "math":
        return "math"
    if intent == "faq":
        return "faq"
    if intent == "search":
        return "search"
    return "chitchat"

graph.set_entry_point("classify")
graph.add_conditional_edges("classify", route_from_intent)
graph.add_edge("math", "compose")
graph.add_edge("faq", "compose")
graph.add_edge("search", "compose")
graph.add_edge("chitchat", "compose")
graph.set_finish_point("compose")

app = graph.compile()

########################################
# 5) 간단 실행 예시
########################################
if __name__ == "__main__":
    examples = [
        "12*(3+4)/7",                                   # math
        "LangChain 버전 확인 방법 알려줘",                # faq
        "LangGraph가 무엇인지 한 줄로 설명해줘",          # faq
        "RAG 파이프라인 단계 알려줘",                     # faq 또는 search
        "하늘은 왜 파란가요?",                            # chitchat
        "LangChain에서 RAG 구축 핵심 요약"                # search
    ]
    for q in examples:
        out = app.invoke({"question": q})
        print("Q:", q)
        print("— intent:", out.get("intent"), "| reason:", out.get("reason"))
        print(out["final_answer"])
        print("-" * 60)


Q: 12*(3+4)/7
— intent: math | reason: 사칙연산/수치 계산이 포함된 질문입니다.
계산 결과는 12.0이다.
------------------------------------------------------------
Q: LangChain 버전 확인 방법 알려줘
— intent: faq | reason: 설정/버전/개념/가이드에 대한 질문이므로 FAQ로 분류됨.
LangChain 버전을 확인하는 방법은 다음과 같다.

1. Python 환경에서 다음 코드를 실행한다:
   ```python
   import langchain
   print(langchain.__version__)
   ```

이 코드를 통해 현재 설치된 LangChain의 버전을 확인할 수 있다.
------------------------------------------------------------
Q: LangGraph가 무엇인지 한 줄로 설명해줘
— intent: faq | reason: 설정/버전/개념/가이드에 대한 질문이므로 FAQ로 분류됨.
LangGraph는 언어 모델을 기반으로 한 그래프 구조를 활용하여 자연어 처리 작업을 지원하는 도구이다.
------------------------------------------------------------
Q: RAG 파이프라인 단계 알려줘
— intent: faq | reason: RAG 파이프라인의 단계에 대한 설정 및 개념을 묻는 질문이므로 FAQ로 분류됨.
RAG 파이프라인의 단계는 다음과 같다.

1. **질문 입력**: 사용자가 질문을 입력한다.
2. **문서 검색**: 관련 문서를 검색하여 후보 목록을 생성한다.
3. **문서 선택**: 검색된 문서 중에서 가장 관련성이 높은 문서를 선택한다.
4. **답변 생성**: 선택된 문서를 바탕으로 답변을 생성한다.
5. **출력**: 최종 답변을 사용자에게 제공한다.
------------------------------------------

## 고급 질문 유형별 도구 자동 선택 에이전트

In [5]:
#!/usr/bin/env python3
"""
고급 질문 유형별 도구 자동 선택 에이전트
- 실제 API 연동
- 대화 메모리 기능
- 더 정교한 질문 분류
"""

import os
import json
import requests
from typing import List, Dict, Any, Optional
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from ddgs import DDGS
import re

class AdvancedQuestionClassifier:
    """고급 질문 분류기 - 더 정교한 분류와 컨텍스트 이해"""
    
    def __init__(self, openai_api_key: str):
        self.llm = ChatOpenAI(
            model="gpt-4o",  # 더 정확한 분류를 위해 GPT-4 사용
            api_key=openai_api_key,
            temperature=0.1
        )
        
        self.classification_prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 질문을 분석하여 적절한 도구를 선택하는 전문가입니다.

사용 가능한 도구들:
1. weather_tool - 날씨 관련 질문 (지역명 포함)
2. news_search_tool - 뉴스, 뉴스 검색 관련
3. calculator_tool - 수학 계산, 산술 연산
4. web_search_tool - 일반적인 정보 검색, 조사
5. translation_tool - 언어 번역
6. stock_tool - 주식, 투자 관련 정보
7. general_chat - 일반적인 대화, 인사, 감정 표현

질문을 분석하고 다음 정보를 JSON 형식으로 제공하세요:
{
    "tool": "선택된_도구명",
    "confidence": 0.0-1.0,
    "reason": "선택 이유",
    "parameters": {
        "param1": "추출된_값1",
        "param2": "추출된_값2"
    }
}"""),
            ("human", "질문: {question}")
        ])
    
    def classify_question(self, question: str) -> Dict[str, Any]:
        """질문을 분석하여 적절한 도구와 매개변수를 선택"""
        try:
            response = self.llm.invoke(
                self.classification_prompt.format_messages(question=question)
            )
            
            content = response.content.strip()
            
            # JSON 블록 제거
            if '```json' in content:
                start = content.find('```json') + 7
                end = content.find('```', start)
                if end != -1:
                    content = content[start:end].strip()
            elif '```' in content:
                start = content.find('```') + 3
                end = content.find('```', start)
                if end != -1:
                    content = content[start:end].strip()
            
            # JSON 파싱 시도
            try:
                result = json.loads(content)
                return result
            except json.JSONDecodeError:
                # JSON 파싱 실패 시 간단한 패턴 매칭으로 대체
                return self._fallback_classification(question, content)
                
        except Exception as e:
            # 분류 오류 메시지 제거 - 키워드 기반 분류로 대체
            return self._fallback_classification(question, "")
    
    def _fallback_classification(self, question: str, content: str) -> Dict[str, Any]:
        """JSON 파싱 실패 시 간단한 패턴 매칭으로 분류"""
        question_lower = question.lower()
        
        if any(word in question_lower for word in ['날씨', 'weather', '기온', '온도']):
            return {
                "tool": "weather_tool",
                "confidence": 0.8,
                "reason": "날씨 관련 키워드 감지",
                "parameters": {"location": self._extract_location(question)}
            }
        elif any(word in question_lower for word in ['뉴스', 'news', '기사']):
            return {
                "tool": "news_search_tool",
                "confidence": 0.8,
                "reason": "뉴스 관련 키워드 감지",
                "parameters": {"query": question}
            }
        elif any(word in question_lower for word in ['계산', '더하기', '빼기', '곱하기', '나누기', '+', '-', '*', '/']):
            return {
                "tool": "calculator_tool",
                "confidence": 0.8,
                "reason": "계산 관련 키워드 감지",
                "parameters": {"expression": self._extract_expression(question)}
            }
        elif any(word in question_lower for word in ['검색', '찾아', '알려', '정보']):
            return {
                "tool": "web_search_tool",
                "confidence": 0.7,
                "reason": "검색 관련 키워드 감지",
                "parameters": {"query": question}
            }
        elif any(word in question_lower for word in ['번역', 'translate', '영어', 'english']):
            return {
                "tool": "translation_tool",
                "confidence": 0.8,
                "reason": "번역 관련 키워드 감지",
                "parameters": {"text": self._extract_translation_text(question), "target_language": "영어"}
            }
        elif any(word in question_lower for word in ['주식', 'stock', '주가', '투자']):
            return {
                "tool": "stock_tool",
                "confidence": 0.8,
                "reason": "주식 관련 키워드 감지",
                "parameters": {"symbol": "AAPL"}
            }
        else:
            return {
                "tool": "general_chat",
                "confidence": 0.5,
                "reason": "일반 대화로 분류",
                "parameters": {}
            }
    
    def _extract_expression(self, question: str) -> str:
        """질문에서 수학 표현식 추출"""
        import re
        # 더 정확한 수학 표현식 추출
        # 숫자, 연산자, 괄호, 공백을 포함한 패턴
        pattern = r'[\d\+\-\*\/\(\)\.\s]+'
        matches = re.findall(pattern, question)
        
        if matches:
            # 가장 긴 매치를 선택 (더 완전한 표현식)
            longest_match = max(matches, key=len).strip()
            # 공백 제거
            expression = longest_match.replace(' ', '')
            return expression
        
        # 패턴 매칭이 실패하면 질문에서 숫자와 연산자만 추출
        expression = re.sub(r'[^\d\+\-\*\/\(\)\.]', '', question)
        return expression if expression else question
    
    def _extract_location(self, question: str) -> str:
        """질문에서 지역명 추출"""
        # 한국 도시들
        korean_cities = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '수원', '성남', '고양', '용인', '청주', '천안', '전주', '포항', '제주']
        
        # 해외 도시들
        international_cities = ['뉴욕', 'New York', '런던', 'London', '도쿄', 'Tokyo', '베이징', 'Beijing', '파리', 'Paris', '시드니', 'Sydney', '로스앤젤레스', 'Los Angeles', 'LA']
        
        # 모든 도시 목록
        all_locations = korean_cities + international_cities
        
        for location in all_locations:
            if location in question:
                return location
        
        return "서울"  # 기본값
    
    def _extract_translation_text(self, question: str) -> str:
        """질문에서 번역할 텍스트 추출"""
        import re
        
        # "XXX를 영어로 번역해줘" 패턴에서 텍스트 추출
        patterns = [
            r'(.+?)를\s*영어로\s*번역',
            r'(.+?)을\s*영어로\s*번역',
            r'(.+?)를\s*english로\s*번역',
            r'(.+?)을\s*english로\s*번역',
            r'(.+?)를\s*번역',
            r'(.+?)을\s*번역'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, question)
            if match:
                text = match.group(1).strip()
                # 불필요한 단어 제거
                text = text.replace('을', '').replace('를', '').strip()
                return text
        
        # 패턴 매칭이 실패하면 질문 전체에서 한글만 추출
        korean_text = re.findall(r'[가-힣]+', question)
        if korean_text:
            return korean_text[0]
        
        return question

# API 키 없이 사용 가능한 도구들
@tool
def weather_tool(location: str) -> str:
    """실제 날씨 정보를 제공합니다."""
    try:
        # wttr.in 무료 날씨 API 사용 (API 키 불필요)
        url = f"https://wttr.in/{location}?format=%C+%t+%h+%w+%p"
        response = requests.get(url, timeout=10)
        
        if response.status_code == 200:
            weather_data = response.text.strip()
            return f"{location}의 현재 날씨: {weather_data}"
        else:
            return f"{location}의 날씨 정보를 가져올 수 없습니다."
            
    except Exception as e:
        return f"날씨 정보 조회 중 오류 발생: {e}"

@tool
def news_search_tool(query: str) -> str:
    """실제 뉴스 검색 결과를 제공합니다."""
    try:
        # DuckDuckGo를 사용한 뉴스 검색 (API 키 불필요)
        with DDGS() as ddgs:
            results = list(ddgs.news(query, max_results=5))
            
            if results:
                news_text = f"'{query}' 관련 최신 뉴스:\n"
                for i, article in enumerate(results, 1):
                    title = article.get('title', '제목 없음')
                    url = article.get('url', '')
                    news_text += f"{i}. {title}\n   링크: {url}\n"
                return news_text
            else:
                return f"'{query}' 관련 뉴스를 찾을 수 없습니다."
                
    except Exception as e:
        return f"뉴스 검색 중 오류 발생: {e}"

@tool
def calculator_tool(expression: str) -> str:
    """안전한 수학 계산을 수행합니다."""
    try:
        # 허용된 수학 연산자와 함수만 사용
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "허용되지 않는 문자가 포함되어 있습니다."
        
        # 위험한 함수들 제거
        dangerous_functions = ['eval', 'exec', 'import', '__']
        if any(func in expression.lower() for func in dangerous_functions):
            return "보안상 허용되지 않는 함수가 포함되어 있습니다."
        
        result = eval(expression)
        return f"계산 결과: {expression} = {result}"
    except Exception as e:
        return f"계산 오류: {e}"

@tool
def web_search_tool(query: str) -> str:
    """웹 검색 결과를 제공합니다."""
    try:
        # DuckDuckGo를 사용한 웹 검색 (API 키 불필요)
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=5))
            
            if results:
                search_text = f"'{query}' 검색 결과:\n"
                for i, result in enumerate(results, 1):
                    title = result.get('title', '제목 없음')
                    body = result.get('body', '내용 없음')
                    url = result.get('href', '')
                    search_text += f"{i}. {title}\n   내용: {body[:100]}...\n   링크: {url}\n\n"
                return search_text
            else:
                return f"'{query}' 관련 정보를 찾을 수 없습니다."
                
    except Exception as e:
        return f"웹 검색 중 오류 발생: {e}"

@tool
def translation_tool(text: str, target_language: str = "영어") -> str:
    """텍스트를 다른 언어로 번역합니다."""
    try:
        # 확장된 번역 사전
        translations = {
            # 인사말
            "안녕하세요": "Hello",
            "안녕": "Hi",
            "안녕히가세요": "Goodbye",
            "안녕히계세요": "Goodbye",
            "좋은아침": "Good morning",
            "좋은저녁": "Good evening",
            "좋은밤": "Good night",
            
            # 감사/사과
            "감사합니다": "Thank you",
            "고마워요": "Thank you",
            "고맙습니다": "Thank you",
            "미안해요": "I'm sorry",
            "죄송합니다": "I'm sorry",
            "괜찮아요": "It's okay",
            
            # 감정
            "사랑해요": "I love you",
            "좋아해요": "I like you",
            "행복해요": "I'm happy",
            "슬퍼요": "I'm sad",
            "화나요": "I'm angry",
            "기뻐요": "I'm glad",
            
            # 도움/요청
            "도와주세요": "Please help me",
            "도와줘": "Help me",
            "부탁해요": "Please",
            "제발": "Please",
            
            # 일상
            "잘지내세요": "Take care",
            "건강하세요": "Stay healthy",
            "조심하세요": "Be careful",
            "화이팅": "Fighting",
            "힘내세요": "Cheer up",
            
            # 음식
            "맛있어요": "It's delicious",
            "배고파요": "I'm hungry",
            "목말라요": "I'm thirsty",
            
            # 날씨
            "날씨가좋아요": "The weather is nice",
            "비가와요": "It's raining",
            "눈이와요": "It's snowing",
            "춥다": "It's cold",
            "덥다": "It's hot",
            
            # 숫자
            "하나": "one",
            "둘": "two", 
            "셋": "three",
            "넷": "four",
            "다섯": "five",
            "여섯": "six",
            "일곱": "seven",
            "여덟": "eight",
            "아홉": "nine",
            "열": "ten"
        }
        
        # 텍스트 정리 (공백 제거)
        clean_text = text.replace(' ', '').replace('\n', '')
        
        if clean_text in translations:
            return f"'{text}'를 {target_language}로 번역: {translations[clean_text]}"
        else:
            # 부분 매칭 시도
            for korean, english in translations.items():
                if korean in clean_text:
                    return f"'{text}'에서 '{korean}'를 {target_language}로 번역: {english}"
            
            return f"'{text}'의 번역을 찾을 수 없습니다. 더 정교한 번역을 위해서는 번역 API가 필요합니다."
            
    except Exception as e:
        return f"번역 중 오류 발생: {e}"

@tool
def stock_tool(symbol: str) -> str:
    """주식 정보를 제공합니다."""
    try:
        # 간단한 주식 정보 (API 키 없이 사용 가능한 기본 정보)
        stock_info = {
            "AAPL": "Apple Inc. - 기술 주식",
            "GOOGL": "Alphabet Inc. - 구글 모회사",
            "MSFT": "Microsoft Corporation - 마이크로소프트",
            "TSLA": "Tesla Inc. - 전기차 회사",
            "AMZN": "Amazon.com Inc. - 아마존",
            "META": "Meta Platforms Inc. - 페이스북",
            "NVDA": "NVIDIA Corporation - 반도체 회사"
        }
        
        if symbol.upper() in stock_info:
            return f"{symbol.upper()} 주식 정보: {stock_info[symbol.upper()]}\n실시간 가격 정보를 위해서는 주식 API가 필요합니다."
        else:
            return f"{symbol} 주식 정보를 찾을 수 없습니다. 지원되는 주식: {', '.join(stock_info.keys())}"
            
    except Exception as e:
        return f"주식 정보 조회 중 오류 발생: {e}"

class AdvancedToolSelectorAgent:
    """고급 질문 유형별 도구 자동 선택 에이전트"""
    
    def __init__(self, openai_api_key: str):
        self.classifier = AdvancedQuestionClassifier(openai_api_key)
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            api_key=openai_api_key,
            temperature=0.3
        )
        
        # 도구 매핑
        self.tools = {
            "weather_tool": weather_tool,
            "news_search_tool": news_search_tool,
            "calculator_tool": calculator_tool,
            "web_search_tool": web_search_tool,
            "translation_tool": translation_tool,
            "stock_tool": stock_tool
        }
        
        # 일반 대화용 프롬프트
        self.chat_prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 도움이 되는 AI 어시스턴트입니다. 
사용자의 질문에 친절하고 정확하게 답변해주세요."""),
            ("human", "{question}")
        ])
    
    def process_question(self, question: str) -> Dict[str, Any]:
        """질문을 처리하고 적절한 도구를 사용하여 답변을 생성"""
        
        # 1. 질문 유형 분류
        classification = self.classifier.classify_question(question)
        selected_tool = classification["tool"]
        confidence = classification["confidence"]
        reason = classification["reason"]
        parameters = classification["parameters"]
        
        print(f"선택된 도구: {selected_tool} (신뢰도: {confidence:.2f})")
        print(f"선택 이유: {reason}")
        
        # 2. 도구 실행 또는 일반 대화
        if selected_tool in self.tools and confidence > 0.6:
            try:
                tool_func = self.tools[selected_tool]
                
                # 매개변수 준비
                if selected_tool == "weather_tool":
                    location = parameters.get("location", "서울")
                    result = tool_func.invoke({"location": location})
                elif selected_tool == "news_search_tool":
                    query = parameters.get("query", question)
                    result = tool_func.invoke({"query": query})
                elif selected_tool == "calculator_tool":
                    expression = parameters.get("expression", question)
                    result = tool_func.invoke({"expression": expression})
                elif selected_tool == "web_search_tool":
                    query = parameters.get("query", question)
                    result = tool_func.invoke({"query": query})
                elif selected_tool == "translation_tool":
                    text = parameters.get("text", question)
                    target_lang = parameters.get("target_language", "영어")
                    result = tool_func.invoke({"text": text, "target_language": target_lang})
                elif selected_tool == "stock_tool":
                    symbol = parameters.get("symbol", "AAPL")
                    result = tool_func.invoke({"symbol": symbol})
                
                return {
                    "tool_used": selected_tool,
                    "confidence": confidence,
                    "reason": reason,
                    "result": result,
                    "success": True
                }
                
            except Exception as e:
                error_msg = f"도구 실행 중 오류 발생: {e}"
                
                return {
                    "tool_used": selected_tool,
                    "confidence": confidence,
                    "reason": reason,
                    "result": error_msg,
                    "success": False
                }
        else:
            # 일반 대화
            try:
                response = self.llm.invoke(
                    self.chat_prompt.format_messages(question=question)
                )
                
                return {
                    "tool_used": "general_chat",
                    "confidence": confidence,
                    "reason": reason,
                    "result": response.content,
                    "success": True
                }
            except Exception as e:
                error_msg = f"대화 중 오류 발생: {e}"
                
                return {
                    "tool_used": "general_chat",
                    "confidence": confidence,
                    "reason": reason,
                    "result": error_msg,
                    "success": False
                }
    

def main():
    """메인 실행 함수"""
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        print("❌ OPENAI_API_KEY 환경 변수를 설정해주세요.")
        return
    
    agent = AdvancedToolSelectorAgent(openai_api_key)
    
    print("🚀 고급 질문 유형별 도구 자동 선택 에이전트")
    print("=" * 60)
    print("종료: 'quit' | 도움말: 'help'")
    print()
    
    while True:
        try:
            question = input("💬 질문: ").strip()
            
            if question.lower() in ['quit', 'exit', '종료']:
                print("👋 에이전트를 종료합니다.")
                break
            
            if question.lower() == 'help':
                print("""
🔧 사용 가능한 질문 유형:
- 🌤️ 날씨: "서울 날씨 어때?", "뉴욕 날씨는?"
- 📰 뉴스: "최신 뉴스 알려줘", "AI 뉴스 검색해줘"
- 🧮 계산: "2 + 3 * 4 계산해줘", "원의 넓이 구해줘"
- 🔍 검색: "파이썬에 대해 검색해줘", "머신러닝 정보 찾아줘"
- 🌐 번역: "안녕하세요를 영어로 번역해줘"
- 📈 주식: "애플 주식 정보 알려줘", "TSLA 주가 확인해줘"
- 💬 일반 대화: "안녕하세요", "오늘 기분이 어때?"
                """)
                continue
            
            
            if not question:
                continue
            
            print("🤔 분석 중...")
            result = agent.process_question(question)
            
            print(f"\n📋 답변:")
            print(f"🔧 도구: {result['tool_used']}")
            print(f"🎯 신뢰도: {result['confidence']:.2f}")
            print(f"💭 이유: {result['reason']}")
            print(f"📝 결과: {result['result']}")
            print("-" * 60)
            
        except KeyboardInterrupt:
            print("\n👋 에이전트를 종료합니다.")
            break
        except Exception as e:
            print(f"❌ 오류 발생: {e}")

if __name__ == "__main__":
    main()


🚀 고급 질문 유형별 도구 자동 선택 에이전트
종료: 'quit' | 도움말: 'help'



💬 질문:  뉴욕 날씨는?


🤔 분석 중...
선택된 도구: weather_tool (신뢰도: 0.80)
선택 이유: 날씨 관련 키워드 감지

📋 답변:
🔧 도구: weather_tool
🎯 신뢰도: 0.80
💭 이유: 날씨 관련 키워드 감지
📝 결과: 뉴욕의 현재 날씨: Clear +7°C 60% ↙13km/h 0.0mm
------------------------------------------------------------


💬 질문:  안녕하세요를 영어로 번역해줘


🤔 분석 중...
선택된 도구: translation_tool (신뢰도: 0.80)
선택 이유: 번역 관련 키워드 감지

📋 답변:
🔧 도구: translation_tool
🎯 신뢰도: 0.80
💭 이유: 번역 관련 키워드 감지
📝 결과: '안녕하세요'를 영어로 번역: Hello
------------------------------------------------------------


...

🤔 분석 중...
선택된 도구: general_chat (신뢰도: 0.50)
선택 이유: 일반 대화로 분류

📋 답변:
🔧 도구: general_chat
🎯 신뢰도: 0.50
💭 이유: 일반 대화로 분류
📝 결과: 안녕하세요! 어떻게 도와드릴까요?
------------------------------------------------------------


💬 질문:  파이썬에 대해 검색해줘


🤔 분석 중...
선택된 도구: web_search_tool (신뢰도: 0.70)
선택 이유: 검색 관련 키워드 감지

📋 답변:
🔧 도구: web_search_tool
🎯 신뢰도: 0.70
💭 이유: 검색 관련 키워드 감지
📝 결과: '파이썬에 대해 검색해줘' 검색 결과:
1. Step1. 파이썬에 대해 알아보자. - 벨로그
   내용: 목차1. Python 소개2. Python의 장점3. Python의 활용 분야4. Python Editor의 종류 이번 장에서는 파이 썬 에 대한 모든 것, 활용분야, 개발 too...
   링크: https://velog.io/@justdotheg/1편-Python-프로그래밍-기초-학습방법

2. 파이썬에 대한 완벽 정리: 시간은 없지만, 배우고 싶은 당신을 위하여
   내용: Jun 29, 2023 · 파이 썬 에 대해 공부할수록, 모르는 지식이 나오고 그 지식을 찾기 위해 계속 시간을 투자해야 하니까요. 이 글은 바쁘지만, 배우고 싶은 분들을 위해 파...
   링크: https://kevinitcoding.tistory.com/entry/파이썬에-대한-완벽-정리-시간은-없지만-배우고-싶은-당신을-위하여

3. 파이썬에 대한 기본적인 설명 및 개발도구와 라이브러리 | LeeSponse
   내용: Apr 4, 2024 · Home / Python / 파이 썬 에 대한 기본적인 설명 및 개발도구와 라이브러리...
   링크: https://leesponse.github.io/python/Python/

4. 파이썬에 대해 알아보자! - 네이버 블로그
   내용: 일상 파이 썬 에 대해 알아보자! 현재 ・ 35분 전 URL 복사 이웃추가 파이 썬 의 탄생과 특징...
   링크: https://blog.naver.com/ju_present/223984981634

5. 파이썬 (Python) 바로 알기 – 특징, 장점, 활용 사례 | 가비아 라이브...
   내용: 아래에서는 파이 썬 의 특징 및 장점, 그리고 활용 사례에

💬 질문:  quit


👋 에이전트를 종료합니다.
